# Map georeferencing notebook - Python

In [1]:
import urllib.request
import pandas as pd
import geopandas
import geopy
from geopy.geocoders import Nominatim
import os
import folium
import math
from folium import Choropleth, Circle, Marker
from folium.plugins import MarkerCluster
import geopandas as gpd
import numpy as np

In [2]:
# url = "https://raw.githubusercontent.com/AngelusGi/UniMolFinalExam/main/Dataset/Original_CSV/Angelo/trackLog-2019-dic-23_13-53-06.csv"
# file_name = "trackLog-2019-dic-23_13-53-06.csv"

In [3]:
# urllib.request.urlretrieve(url, filename=file_name)

Questo metodo elimina dal corrente dataframe tutte le colonne, eccezion fatta per latitudine e longitudine

In [4]:
def clean_header(file_to_process):
    df = pd.read_csv(file_to_process)
    keeps_only_latidue_longitude = [ "GPS Time","Device Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]

    gps_long_lat = df.drop(columns=keeps_only_latidue_longitude)
    return gps_long_lat

In [5]:
def add_csv_to_df(file_to_process):
    return pd.read_csv(file_to_process)

prede tutti i file nella cartella e li trasforma in un dizionario di dataframe

In [6]:
# verificare csv_path
basepath = os.environ['USERPROFILE']
csv_path = r"{}\GitHub\UniMolFinalExam\Notebooks\Python\TestSet".format(basepath)

In [7]:
coordinates_data = dict()
full_data = dict()

In [8]:
for directory, subdir_list, file_list in os.walk(csv_path):
    for file_name in file_list:
        print('File:', file_name)
        path_to_read_csv = csv_path + "\\" + file_name
        print(path_to_read_csv)

        temp_dataframe = clean_header(path_to_read_csv)
        temp_file_name_ext = file_name.replace('.csv','')
        final_file_name = temp_file_name_ext.replace('trackLog-','')
        coordinates_data[final_file_name] = temp_dataframe

        full_data[final_file_name] = add_csv_to_df(path_to_read_csv)
        print()
    print()

File: Test.csv
C:\Users\angel\GitHub\UniMolFinalExam\Notebooks\Python\TestSet\Test.csv




crea un dizionario contenete solo le coordinate di latitudine e longitudine

In [9]:
coordinates_data

{'Test':      Longitude   Latitude
 0    13.926052  42.050242
 1    13.926052  42.050242
 2    13.926052  42.050242
 3    13.926060  42.050236
 4    13.926061  42.050237
 ..         ...        ...
 848  13.879728  42.114948
 849  13.879526  42.114965
 850  13.879237  42.114989
 851  13.879002  42.115060
 852  13.878757  42.115144
 
 [853 rows x 2 columns]}

In [10]:
coordinates_data["Test"].head()

,Longitude,Latitude
0,13.926052,42.050242
1,13.926052,42.050242
2,13.926052,42.050242
3,13.926060,42.050236
4,13.926061,42.050237


estrae un singolo dataframe

In [11]:
gps_long_lat = coordinates_data["Test"]

In [12]:
gps_long_lat

,Longitude,Latitude
0,13.926052,42.050242
1,13.926052,42.050242
2,13.926052,42.050242
3,13.926060,42.050236
4,13.926061,42.050237
...,...,...
848,13.879728,42.114948
849,13.879526,42.114965
850,13.879237,42.114989
851,13.879002,42.115060


ottenere un indirizzo a partire dalle coordinate

In [13]:
locator = Nominatim(user_agent="agvGeoCoder")

In [14]:
# gps_address = locator.geocode("42.048168,13.931077")

In [15]:
coordinate = gps_long_lat.iloc[0]

In [16]:
gps_address = locator.geocode("{},{}".format(coordinate["Latitude"],coordinate["Longitude"]))

In [17]:
print("Location = {}".format(gps_address))
gps_address.point

Location = Via Giovanni Pansa, Zappannotte, Marane, Sulmona, L'Aquila, Abruzzo, 67039, Italia


Point(42.0500487, 13.9256916, 0.0)

In [18]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="UniMolFinalExam")

In [19]:
place_to_find = "Italy"
geo_cordinates = geolocator.geocode(place_to_find)
geo_cordinates

Location(Italia, (42.6384261, 12.674297, 0.0))

In [20]:
# map_dimension = 650
# map_zoom = 6

# italy_map = folium.Map(location=[geo_cordinates.latitude,geo_cordinates.longitude], zoom_start=map_zoom, width=map_dimension, height=map_dimension)

In [21]:
gps_address

Location(Via Giovanni Pansa, Zappannotte, Marane, Sulmona, L'Aquila, Abruzzo, 67039, Italia, (42.0500487, 13.9256916, 0.0))

In [22]:
gps_address.latitude

42.0500487

In [23]:
italy_map = folium.Map(location=[gps_address.latitude,gps_address.longitude])

In [24]:
# italy_map

In [25]:
for index in range(len(gps_long_lat)):
    position = gps_long_lat.iloc[index]
    
    folium.Circle(
      location=[position["Latitude"],position["Longitude"]],
      radius = 20,
      popup= "test",
      color= 'crimson',
      fill= True,
      fill_color = 'crimson'
   ).add_to(italy_map)

In [26]:
italy_map


esporta la mappa prodotta in un file html

In [27]:
italy_map.save("TestMap.html")